In [1]:
from pyMSOO.MFEA.model import SBSGA
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.utils.DimensionAwareStrategy import *
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 

In [2]:
import numpy as np
import random
from functools import reduce
import numba as nb
import copy

from pyMSOO.MFEA.model import AbstractModel
from pyMSOO.utils import Crossover, Mutation, Selection, DimensionAwareStrategy
from pyMSOO.utils.EA import *
from pyMSOO.utils.numba_utils import numba_randomchoice

class model(AbstractModel.model):
    def compile(self, 
        IndClass: Type[Individual],
        tasks: List[AbstractTask], 
        crossover: Crossover.SBX_Crossover, 
        mutation: Mutation.PolynomialMutation, 
        selection: Selection.ElitismSelection,
        dimension_strategy: DimensionAwareStrategy.AbstractDaS = DimensionAwareStrategy.NoDaS(),
        *args, **kwargs):
        super().compile(IndClass, tasks, crossover, mutation,dimension_strategy, selection, *args, **kwargs)
    
    def fit(self, nb_generations, B = 0.25, H = 0.5, nb_inds_each_task = 100, evaluate_initial_skillFactor = True, *args, **kwargs) -> List[Individual]:
        super().fit(*args, **kwargs)

        # initialize population
        self.population = Population(
            self.IndClass,
            nb_inds_tasks = [nb_inds_each_task] * len(self.tasks), 
            dim = self.dim_uss,
            list_tasks= self.tasks,
            evaluate_initial_skillFactor = evaluate_initial_skillFactor
        )
        self.B = B
        self.H = H

        self.M = np.ones((len(self.tasks), len(self.tasks)))
        self.N = np.ones((len(self.tasks), len(self.tasks)))
        self.C = np.ones((len(self.tasks), len(self.tasks)))
        self.O = np.ones((len(self.tasks), len(self.tasks)))
        self.P = np.ones((len(self.tasks), len(self.tasks)))
        self.A = np.ones((len(self.tasks), len(self.tasks)))
        self.R = np.ones((len(self.tasks), len(self.tasks)))
        
        # save history
        self.history_cost.append([ind.fcost for ind in self.population.get_solves()])
        
        self.render_process(0, ['Cost'], [self.history_cost[-1]], use_sys= True)

        for epoch in range(nb_generations):

            offsprings = Population(
                self.IndClass,
                nb_inds_tasks = [0] * len(self.tasks), 
                dim = self.dim_uss,
                list_tasks= self.tasks,
            )

            copy_offsprings = Population(
                self.IndClass,
                nb_inds_tasks = [0] * len(self.tasks), 
                dim = self.dim_uss,
                list_tasks= self.tasks,
            )

            subPop = [0] * len(self.tasks)
            other_task = [0] * len(self.tasks)

            # Generate offsprings
            for i in range(len(self.tasks)):
                offs = Population(
                    self.IndClass,
                    nb_inds_tasks = [0] * len(self.tasks), 
                    dim = self.dim_uss,
                    list_tasks= self.tasks,
                )
                while len(offs) < nb_inds_each_task:
                    # choose parent 
                    pa, pb = self.population[i].__getRandomItems__(2)
                    # intra crossover
                    oa, ob = self.crossover(pa, pb, i, i)
                    # mutate
                    oa = self.mutation(oa, return_newInd= True)
                    oa.skill_factor = pa.skill_factor

                    ob = self.mutation(ob, return_newInd= True)    
                    ob.skill_factor = pb.skill_factor
                    
                    offs.__addIndividual__(oa)
                    offs.__addIndividual__(ob)

                offsprings += offs
                copy_offsprings += offs
            
            # Inter task

            # np.fill_diagonal(self.R, -1)
            # R_i = np.max(self.R, axis = 1)
            # task_i = list(np.where(np.random.rand() < R_i)[0])
            # task_j = list(np.argmax(self.R[task_i], axis = 1))
            # # S_i = np.min(np.max((R_i[t] * nb_inds_each_task).astype(int), 0), nb_inds_each_task)
            # S_i = np.clip((R_i[task_i] * nb_inds_each_task).astype(int), 0, nb_inds_each_task)
            # # print(task_j)
            # # print(S_i)
            # S_i = np.clip((R_i[task_i] * nb_inds_each_task).astype(int), 0, nb_inds_each_task)
            # # subPop[t] = [self.IndClass(off.genes, skill_factor=i, fcost=t(off.genes)) for off in copy_offsprings[task_j][0: S_i]]
            # # subPop = [[self.IndClass(off.genes, skill_factor=i, fcost=self.tasks[i](off.genes)) for off in copy_offsprings[i][0: S_i[i]]] for i in task_j]
            # # offsprings[t][nb_inds_each_task - S_i: nb_inds_each_task] = subPop[t]
            # for i, j in zip(task_i, task_j):
            #     subPop[i] = [self.IndClass(off.genes, skill_factor=i, fcost=self.tasks[i](off.genes)) for off in copy_offsprings[j][0: S_i[i]]]
            #     offsprings[i][nb_inds_each_task - S_i[i]: nb_inds_each_task] = subPop[i]
            # offsprings.update_rank()
            # copy_offsprings.update_rank()

            for i, t in enumerate(self.tasks):
                self.R[i][i] = -1
                R_i = max(self.R[i])
                if random.random() < R_i:
                    
                    task_j = np.argmax(self.R[i])
                    S_i = min(max(int(R_i * nb_inds_each_task), 0), nb_inds_each_task)
                    other_task[i] = task_j

                    subPop[i] = [self.dimension_strategy(off, task_j, self.population[i].__getRandomItems__()) 
                                 for off in copy_offsprings[task_j][0: S_i]]
                    subPop[i] = [self.IndClass(off.genes, skill_factor=i, fcost = t(off.genes)) for off in subPop[i]]
                    
                    offsprings[i][nb_inds_each_task - S_i: nb_inds_each_task] = subPop[i]
                    
                    # tmp = []
                    # for k in np.where(copy_offsprings[task_j].factorial_rank <= S_i)[0]:
                    #     off = copy_offsprings[task_j][k]
                    #     tmp.append(self.dimension_strategy(off, task_j, self.population[i].__getRandomItems__()))
                    
                    # subPop[i] = tmp
                    # # subPop[i] = [self.dimension_strategy(off, task_j, self.population[i].__getRandomItems__()) 
                    # #              for off in [copy_offsprings[task_j][k] for k in np.where(copy_offsprings[task_j].factorial_rank < S_i)[0]]]
                    # subPop[i] = [self.IndClass(off.genes, skill_factor=i, fcost = t(off.genes)) for off in subPop[i]]
                    
                    # for idx ,k in enumerate(np.where(offsprings[i].factorial_rank > nb_inds_each_task - S_i)[0]):
                    #     offsprings[i][k] = subPop[i][idx]

            # merge and update rank
            self.population = self.population + offsprings
            self.population.update_rank()

            # selection
            self.selection(self.population, [nb_inds_each_task] * len(self.tasks))

            # update symbiosis and rate
            self.update_symbiosis(subPop, other_task)
            # self.update_rate()
            self.R = self.__class__.update_rate(self.M, self.O, self.P, self.A, self.C, len(self.tasks))

            # update operators
            self.crossover.update(population = self.population)
            self.mutation.update(population = self.population)
            self.dimension_strategy.update(population = self.population)

            # save history
            self.history_cost.append([ind.fcost for ind in self.population.get_solves()])

            #print
            self.render_process((epoch+1)/nb_generations, ['Cost'], [self.history_cost[-1]], use_sys= True)
        
        print('\nEND!')

        #solve 
        self.last_pop = self.population
        return self.last_pop.get_solves()

    # def update_rate(self):
    #     for t in range(len(self.tasks)):
    #         T_pos = self.M[t] + self.O[t] + self.P[t]
    #         T_neg = self.A[t] + self.C[t]
    #         T_neu = self.M[t]
    #         self.R[t] = T_pos/(T_pos + T_neg + T_neu)
    #         self.R[t][t] = -1

    @jit(nopython = True)
    def update_rate(M, O, P, A, C, nb_tasks):
        R = np.zeros((nb_tasks, nb_tasks))
        for t in range(nb_tasks):
            T_pos = M[t] + O[t] + P[t]
            T_neg = A[t] + C[t]
            T_neu = M[t]
            R[t] = T_pos/(T_pos + T_neg + T_neu)
            R[t][t] = -1
        return R

    def update_symbiosis(self, subpop: List[Individual], other_task):
        for i in range(len(self.tasks)):
            if isinstance(subpop[i], list):
                for o in subpop[i]:
                    j = other_task[i]
                    if i != j:
                        r_i = self.rank(o, i)
                        r_j = self.rank(o, j)
                        # r_i = self.__class__.rank(self.tasks[i](o), nb.typed.List(self.population[i].getFitness()))
                        # r_j = self.__class__.rank(self.tasks[j](o), nb.typed.List(self.population[j].getFitness()))
                        if (self.isBenefit(r_i) and self.isBenefit(r_j)):
                            self.M[i][j] += 1
                        elif (self.isNeural(r_i) and self.isNeural(r_j)):
                            self.N[i][j] += 1
                        elif (self.isHarmful(r_i) and self.isHarmful(r_j)):
                            self.C[i][j] += 1
                        elif (self.isBenefit(r_i) and self.isNeural(r_j)):
                            self.O[i][j] += 1
                        elif (self.isBenefit(r_i) and self.isHarmful(r_j)): 
                            self.P[i][j] += 1
                        elif (self.isNeural(r_i) and self.isHarmful(r_j)):
                            self.A[i][j] += 1

    def rank(self, off: Individual, task):
        fitness = self.tasks[task](off)
        rank = np.searchsorted(self.population[task].getFitness(), fitness)
        return rank/len(self.population[task])
    
    # @jit(nopython = True)
    # def rank(off_fitness, subPop_fitness):
    #     # fitness = self.tasks[task](off)
    #     # print(self.population[task].getFitness())
    #     rank = np.searchsorted(subPop_fitness, off_fitness)
    #     return rank/len(subPop_fitness)
    
    def isBenefit(self, r):
        if (r <= self.B):
            return True
        return False

    def isHarmful(self, r) :
        if (r > self.H) :
            return True
        return False

    def isNeural(self, r) :
        return (r > self.B and r <= self.H)

In [3]:
# tasks, IndClass = CEC17_benchmark.get_2tasks_benchmark(1)
# tasks, IndClass = WCCI22_benchmark.get_complex_benchmark(1)
tasks, IndClass = CEC17_benchmark.get_10tasks_benchmark()

In [4]:
baseModel = SBSGA.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy=DaS_strategy(eta= 3)
)
solve = baseModel.fit(
    nb_generations = 1000, rmp = 0.5, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Seed: None -- Time: 01m 34.59s  100 % [====================>]  Cost: 3.50E-06  5.89E-07  1.39E-16  0.00E+00  4.68E-04  1.90E-04  5.47E-07  6.38E-04  6.84E-14  3.98E+00  ,  
END!


### MUltitime Model

In [26]:
baseModel = MultiTimeModel(model= model)
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy= NoDaS(),
)
solve = baseModel.fit(
    nb_generations = 1000,nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

In [27]:
baseModel.run(
    nb_run= 30,
    # save_path= './RESULTS/MFEA_cec17.mso'
)

Seed: 0 -- Time: 01m 21.29s  100 % [====================>]  Cost: 1.92E-06  2.73E-07  1.79E-10  0.00E+00  2.59E-04  1.66E-04  1.68E-03  6.39E-04  3.48E-08  1.67E+01  ,  
END!
Seed: 1 -- Time: 01m 17.42s  100 % [====================>]  Cost: 1.91E-06  2.89E-07  1.78E-08  0.00E+00  2.51E-04  1.72E-04  7.43E-03  6.38E-04  4.25E-08  2.11E+01  ,  
END!
Seed: 2 -- Time: 01m 18.05s  100 % [====================>]  Cost: 1.10E-06  2.70E-07  1.96E-10  0.00E+00  1.64E-04  1.48E-04  2.02E-03  6.38E-04  2.52E-08  3.47E+01  ,  
END!
Seed: 3 -- Time: 01m 21.33s  100 % [====================>]  Cost: 2.84E-06  3.02E-07  1.18E-10  0.00E+00  4.72E-04  1.69E-04  1.61E-03  6.38E-04  4.23E-08  2.30E+01  ,  
END!
Seed: 4 -- Time: 01m 13.80s   92 % [==================> ]  Cost: 9.49E-06  1.69E-06  4.16E-09  0.00E+00  1.12E-03  3.76E-04  3.97E-03  6.49E-04  9.89E-08  2.15E+01  ,  

f:\BTVN\DSAI\Optimization Lab\pyMSOO\pyMSOO\utils\MultiRun\RunMultiTime.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.array(result[:][:min([len(his) for his in result])][:])


functions : <generator object stream_list at 0x0000015A6ECA8900>, <class 'generator'> cannot saved in process_save_dict function
attributes : <generator object stream_list at 0x0000015A6ECA8970>, <class 'generator'> cannot saved in process_save_dict function
globals : <generator object stream_list at 0x0000015A6ECA89E0>, <class 'generator'> cannot saved in process_save_dict function
functions : <generator object stream_list at 0x0000015A6ECA8A50>, <class 'generator'> cannot saved in process_save_dict function
attributes : <generator object stream_list at 0x0000015A6ECA8AC0>, <class 'generator'> cannot saved in process_save_dict function
globals : <generator object stream_list at 0x0000015A6ECA8B30>, <class 'generator'> cannot saved in process_save_dict function
functions : <generator object stream_list at 0x0000015A6ECA8BA0>, <class 'generator'> cannot saved in process_save_dict function
attributes : <generator object stream_list at 0x0000015A6ECA8C10>, <class 'generator'> cannot saved

: 